```c
#if __GNUC__ >= 4
#ifdef __EXPORT
#undef __EXPORT
#endif
#define __EXPORT __attribute__((visibility("default")))
#ifdef __PRIVATE
#undef __PRIVATE
#endif
#define __PRIVATE __attribute__((visibility("hidden")))
#else
#define __EXPORT
#define __PRIVATE
#endif
```

In [1]:
import yaml
from jinja2 import Environment
from jinja2.loaders import FileSystemLoader
# import re
from pathlib import Path
from collections import namedtuple
# import logging
from colorama import Fore
from pprint import pprint
import struct
import enum

In [3]:
PrimType = namedtuple("PrimType","type size var")
Enum = namedtuple("Enum","name size values")
Constant = namedtuple("Constant","type var value")

# tmp_dir = pathlib.Path(__file__).resolve().parent/"templates"
# env = Environment(loader=FileSystemLoader(tmp_dir))
tmp_dir = Path("templates").resolve()
env = Environment(loader=FileSystemLoader(tmp_dir))
tmpl = env.get_template("msg.cpp.jinja")

class Lang(enum.Enum):
    python = 1
    c = 2

In [4]:
KEYWORDS = ["enum","define","namespace","import","id","name"]

# c - C/C++ name
# py - python name
# fmt - python struct pack/unpack
# size - number of bytes
# complex - for user defined types, more involved
VarInfo = namedtuple("VarInfo","c py size fmt complex")

var_types = {
    "uint8": VarInfo("uint8_t", "int",1, "B", False),
    "uint16": VarInfo("uint16_t", "int",2, "H", False),
    "uint32": VarInfo("uint32_t", "int", 4, "I", False),
    "uint64": VarInfo("uint64_t", "int",8, "Q", False),
    "int8": VarInfo("int8_t", "int",1, "b", False),
    "int16": VarInfo("int16_t", "int", 2, "h", False),
    "int32": VarInfo("int32_t", "int", 4, "i", False),
    "int64": VarInfo("int64_t", "int", 8, "q", False),
    "float32": VarInfo("float", "float", 4, "f", False),
    "float64": VarInfo("double", "float", 8, "d", False),
    "bool": VarInfo("bool", "bool", 1, "?", False),
    # "vec": VarInfo("vec_t", "vec_t", 12, "3f", True)
}

PRIMITIVE_TYPES = list(var_types.keys())

In [5]:
from colorama import Fore


class MsgParts:
    """
    Breaks a message format appart and stores the results so it can be
    converted into other languages. Supported languages:
    - python
    - C/C++
    """
    def __init__(self):
        # self.comments = []  # comments in body of message prototype
        self.fields = []    # variables in message
        self.includes = []  # included message headers/modules
        self.constants = [] # defines
        # self.c_funcs = []   # custom C functions
        # self.py_funcs = []  # custom Python functions
        self.enums = []     # enums
        self.msg_size = 0   # size of message in bytes
        self.name = None    # filename for naming the message
        self.id = 0         # message id number
        self.fmt = None     # struct format string
        # self.namespace = None # cpp namespace

    def get_info(self):
        info = {
            # "name": self.file.stem,
            "name": self.name,
            "vars": self.fields,
            "includes": self.includes,
            "msg_size": self.msg_size,
            "constants": self.constants,
            # "msg_size_type": "uint8_t",
            # "comments": comments,
            # "args": func_args,
            # "functions": msg_parts.c_funcs,
            "enums": self.enums,
            "msgid": self.id,
            "fmt": self.fmt
            # "license_notice": msg_parts.license_notice,
            # "namespace": msg_parts.namespace
        }
        return info

    def __repr__(self):
        return str(self)

    def __str__(self):
        ret = f"{Fore.YELLOW}------------------------------\n"
        ret += f"Name: {self.name}\n"
        ret += f"ID: {self.id}\n"
        ret += f"Size: {self.msg_size} bytes\n"
        ret += f"Fmt: {self.fmt}\n"
        # if self.namespace is not None:
        #     ret += f"Namespace: {self.namespace}\n"
        ret += f"------------------------------\n{Fore.RESET}"
        # ret += f"{Fore.CYAN}Comments:\n{Fore.RESET}"
        # ret += f"{Fore.GREEN}"
        # for c in self.comments:
        #     ret += f" {c}\n"
        # ret += f"{Fore.RESET}"
        
        ret += f"\n{Fore.CYAN}Constants:\n{Fore.RESET}"
        for c in self.constants:
            ret += f" {c}\n"

        ret += f"\n{Fore.CYAN}Fields:\n{Fore.RESET}"
        for f in self.fields:
            ret += f" {f}\n"

        ret += f"\n{Fore.CYAN}Includes:\n{Fore.RESET}"
        ret += f"{Fore.BLUE}"
        for i in self.includes:
            ret += f" {i}\n"
        ret += f"{Fore.RESET}"

        ret += f"\n{Fore.CYAN}Enums:\n{Fore.RESET}"
        for f in self.enums:
            ret += f" {f}\n"

        ret += f"{Fore.CYAN}\nMessage Size:{Fore.RESET}\n"
        ret += f" {self.msg_size} bytes\n"
        ret += f" {self.fmt}\n"
        return ret

In [6]:
def process_enum(value, lang):
    # type = "int32"
    # print(f"enum: {k},{v}")
    name = value["name"]
    type = value.get("type", "int32")
    # if type is not None:
    if lang == Lang.c:
        type = var_types[type].c
    else:
        type = var_types[type].py
    values = []
    for var, val in value["values"].items():
        values.append(f"{name.upper()}_{var.upper()} = {val}")
    e = Enum(name,type,values)
    return e

In [7]:
def process_primative_types(key,value,array=False):
    # print(f"prim: {k},{v}")
    if not isinstance(value, list):
        value = [ value ]
    
    if array:
        start = key.find('[')
        stop = key.find(']')
        size = int(key[start+1:stop])
        dtype = key[:start]
    else:
        dtype = key
        size = 0
        
    ret = []
    for v in value:
        ret.append( PrimType(dtype,size,v) )

    return ret

In [18]:
def read_file(file):
    file_path = Path(file)
    try:
        with file_path.open("r") as file:
            data = yaml.safe_load(file)
        print(data)
    except FileNotFoundError:
        print(f"File {yaml_file} not found")
    except yaml.YAMLError as e:
        print(f"Error parsing YAML: {e}")
        print(data)

    name = file_path.stem
    print(name)
    if name.find('_') > 0:
        info = name.split('_')
        print(info)
        data["name"] = info[1]
        id = info[0]
        data["id"] = int(id)
    else:
        data["name"] = name
        data["id"] = 0
        
    return data

data = read_file("msgs/9_dummy.yml")
print("------------------------------")
pprint(data)

{'package': 'kevin', 'imports': ['vector', 'quat'], 'constants': {'terry': 256.0, 'sam': 128}, 'vector': {'id': 10, 'x': 'float[32]', 'y': 'double', 'z': 'double'}, 'quaternion': {'id': 11, 'x': 'float', 'y': 'float', 'z': 'float', 'w': 'float'}, 'big_bob': {'tom': 'vector', 'data': 'int32[10]', 'bob': 'char[32]'}, 'enums': {'harry': {'a': 0, 'b': 1, 'c': 2}, 'alice': {'go': 1, 'stop': 2, 'continue': 4}, 'paul': ['a', 'b', 'c', 'd']}}
9_dummy
['9', 'dummy']
------------------------------
{'big_bob': {'bob': 'char[32]', 'data': 'int32[10]', 'tom': 'vector'},
 'constants': {'sam': 128, 'terry': 256.0},
 'enums': {'alice': {'continue': 4, 'go': 1, 'stop': 2},
           'harry': {'a': 0, 'b': 1, 'c': 2},
           'paul': ['a', 'b', 'c', 'd']},
 'id': 9,
 'imports': ['vector', 'quat'],
 'name': 'dummy',
 'package': 'kevin',
 'quaternion': {'id': 11,
                'w': 'float',
                'x': 'float',
                'y': 'float',
                'z': 'float'},
 'vector': {'id': 1

In [379]:
def cleanup_fields(fields, lang):
    fmt = ""
    newTypes = []
    for v in fields:
        if v.size > 0:
            fmt += str(v.size) + var_types[v.type].fmt
        else:
            fmt += var_types[v.type].fmt

        if lang == Lang.c:
            newType = var_types[v.type].c
        elif lang == Lang.python:
            newType = var_types[v.type].py
        else:
            raise Exception(f"invalid language: {lang}")
            
        pt = PrimType(newType,v.size,v.var)
        newTypes.append(pt)
    return newTypes, fmt

def parse_msg_data(data, lang):
    parts = MsgParts()
    fields = []
    
    for key, value in data.items():
        if not key.isascii():
            print(f"not ascii: {key}")
            continue
    
        key = key.lower()
            
        if key in PRIMITIVE_TYPES:
            vv = process_primative_types(key, value, array=False)
            for v in vv: fields.append(v)
        elif key.find('[') > 0 and key.find(']'):
            vv = process_primative_types(key, value, array=True)
            for v in vv: fields.append(v)
            
        elif key == "constant":
            if not isinstance(value, list):
                value = [ value ]
            for v in value:
                c = Constant(v["type"], v["name"], v["value"])
                parts.constants.append(c)
                
        elif key in KEYWORDS:
            if key == "enum":
                if not isinstance(value, list):
                    value = [value]
                for v in value:
                    e = process_enum(v, lang)
                    parts.enums.append(e)
        
            elif key == "name":
                parts.name = value
                
            elif key == "import":
                if not isinstance(value, list):
                    value = [ value ]
                for v in value:
                    parts.includes.append(v)
                    
            elif key == "id":
                parts.id = int(value)
                
            else:
                print(f"oops: {key} {value}")
        else:
            print(f"oops need to add: {key} {value}")

    pts, fmt = cleanup_fields(fields, lang)
    for pt in pts:
        parts.fields.append(pt)

    parts.fmt = fmt
    parts.msg_size = struct.calcsize(fmt)

    return parts

parts = parse_msg_data(data, Lang.c)
print(parts)

oops need to add: vec ['accel', 'gyro', 'mag']
------------------------------
Name: test
ID: 8
Size: 72 bytes
Fmt: fffbH5i5i?d
------------------------------

Constants:
 Constant(type='float32', var='terry', value=256)
 Constant(type='uint32', var='fred', value=1024)

Fields:
 PrimType(type='float', size=0, var='timestamp')
 PrimType(type='float', size=0, var='temperature')
 PrimType(type='float', size=0, var='pressure')
 PrimType(type='int8_t', size=0, var='bob')
 PrimType(type='uint16_t', size=0, var='tom')
 PrimType(type='int32_t', size=5, var='barry')
 PrimType(type='int32_t', size=5, var='tom')
 PrimType(type='bool', size=0, var='ok')
 PrimType(type='double', size=0, var='hope')

Includes:
 vec
 tom

Enums:
 Enum(name='harry', size='uint8_t', values=['HARRY_A = 0', 'HARRY_B = 1', 'HARRY_C = 2'])
 Enum(name='sam', size='int32_t', values=['SAM_A = 0', 'SAM_B = 1', 'SAM_C = 2'])

Message Size:
 72 bytes
 fffbH5i5i?d



In [380]:
info = parts.get_info()
pprint(info)

{'constants': [Constant(type='float32', var='terry', value=256),
               Constant(type='uint32', var='fred', value=1024)],
 'enums': [Enum(name='harry', size='uint8_t', values=['HARRY_A = 0', 'HARRY_B = 1', 'HARRY_C = 2']),
           Enum(name='sam', size='int32_t', values=['SAM_A = 0', 'SAM_B = 1', 'SAM_C = 2'])],
 'fmt': 'fffbH5i5i?d',
 'includes': ['vec', 'tom'],
 'msg_size': 72,
 'msgid': 8,
 'name': 'test',
 'vars': [PrimType(type='float', size=0, var='timestamp'),
          PrimType(type='float', size=0, var='temperature'),
          PrimType(type='float', size=0, var='pressure'),
          PrimType(type='int8_t', size=0, var='bob'),
          PrimType(type='uint16_t', size=0, var='tom'),
          PrimType(type='int32_t', size=5, var='barry'),
          PrimType(type='int32_t', size=5, var='tom'),
          PrimType(type='bool', size=0, var='ok'),
          PrimType(type='double', size=0, var='hope')]}


In [381]:
tmpl = env.get_template("msg.cpp.jinja")
content = tmpl.render(info)
print(content)

#pragma once

#include <stdint.h>
#include <stdbool.h>
#include <string>
#include "vec.h"
#include "tom.h"


#ifdef __cplusplus
extern "C" {
#endif

#define TEST_MSG_SIZE 72
#define TEST_MSG_ID 8
#define TEST_TERRY 256
#define TEST_FRED 1024


typedef enum: uint8_t {
  HARRY_A = 0, 
  HARRY_B = 1, 
  HARRY_C = 2
} test_harry_e;



typedef enum: int32_t {
  SAM_A = 0, 
  SAM_B = 1, 
  SAM_C = 2
} test_sam_e;



typedef struct {
  float timestamp;
  float temperature;
  float pressure;
  int8_t bob;
  uint16_t tom;
  int32_t[5] barry;
  
  int32_t[5] tom;
  
  bool ok;
  double hope;

} __attribute__((packed)) test_t;

#ifdef __cplusplus
}
#endif











In [366]:
pprint(var_types)

{'bool': VarInfo(c='bool', py='bool', size=1, fmt='?', complex=False),
 'float32': VarInfo(c='float', py='float', size=4, fmt='f', complex=False),
 'float64': VarInfo(c='double', py='float', size=8, fmt='d', complex=False),
 'int16': VarInfo(c='int16_t', py='int', size=2, fmt='h', complex=False),
 'int32': VarInfo(c='int32_t', py='int', size=4, fmt='i', complex=False),
 'int64': VarInfo(c='int64_t', py='int', size=8, fmt='q', complex=False),
 'int8': VarInfo(c='int8_t', py='int', size=1, fmt='b', complex=False),
 'uint16': VarInfo(c='uint16_t', py='int', size=2, fmt='H', complex=False),
 'uint32': VarInfo(c='uint32_t', py='int', size=4, fmt='I', complex=False),
 'uint64': VarInfo(c='uint64_t', py='int', size=8, fmt='Q', complex=False),
 'uint8': VarInfo(c='uint8_t', py='int', size=1, fmt='B', complex=False)}
